# Задача 1 Права администратора

In [1]:
import functools

user_role = 'dd'

def role_required(func):
    def wrapper(*args, **kwargs):
        global user_role
        if user_role == "admin":
            return func(*args, **kwargs)
        else:
            return "Permission denied"
    return wrapper


In [2]:
#@role_required2(role=user_role)
@role_required
def secret_resource() -> str:
    return "Permission accepted"

In [3]:
user_role = "admin"
print(secret_resource())

Permission accepted


In [4]:
user_role = "user"
print(secret_resource())

Permission denied


# Задача 2 Кэширование

In [5]:
import random
import functools

cache_data = {}

def cache(db: str, expiration: int):
    def decorate_func(func):
        global cache_data
        cache_id = f'{db}:{func.__name__}'
        if cache_id not in cache_data:
            cache_data[cache_id] = {'expiration': expiration, 'data': {}}
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            global cache_data
            if args[0] not in cache_data[cache_id]['data']:
                cache_data[cache_id]['data'][args[0]] = {'expire': 0, 'value': None}
            if cache_data[cache_id]['data'][args[0]]['expire'] == 0:
                cache_data[cache_id]['data'][args[0]]['expire'] = cache_data[cache_id]['expiration']
                cache_data[cache_id]['data'][args[0]]['value'] = func(*args, **kwargs)
                expire = cache_data[cache_id]['data'][args[0]]['expire']
                value = cache_data[cache_id]['data'][args[0]]['value']
                print(f"Info about: {args[0]} from {db}, now cached with expire={expire}. Return value={value}")
            else:
                cache_data[cache_id]['data'][args[0]]['expire'] -= 1
                expire = cache_data[cache_id]['data'][args[0]]['expire']
                value = cache_data[cache_id]['data'][args[0]]['value']
                print(f"Info about: {args[0]} cached in {db}, expire={expire}. Return value={value}")
            return cache_data[cache_id]['data'][args[0]]['value']
        return wrapper
    return decorate_func

def get_info(thing: str) -> str:
    return f'{thing}: {random.randint(1, 100)}'

@cache(db="postgres", expiration=5)
def get_info_postgres(thing: str) -> str:
    return get_info(thing)

@cache(db="sql_lite", expiration=3)
def get_info_sqllite(thing: str) -> str:
    return get_info(thing)

In [6]:
for i in range(10):
    get_info_postgres("bike_store")

Info about: bike_store from postgres, now cached with expire=5. Return value=bike_store: 57
Info about: bike_store cached in postgres, expire=4. Return value=bike_store: 57
Info about: bike_store cached in postgres, expire=3. Return value=bike_store: 57
Info about: bike_store cached in postgres, expire=2. Return value=bike_store: 57
Info about: bike_store cached in postgres, expire=1. Return value=bike_store: 57
Info about: bike_store cached in postgres, expire=0. Return value=bike_store: 57
Info about: bike_store from postgres, now cached with expire=5. Return value=bike_store: 88
Info about: bike_store cached in postgres, expire=4. Return value=bike_store: 88
Info about: bike_store cached in postgres, expire=3. Return value=bike_store: 88
Info about: bike_store cached in postgres, expire=2. Return value=bike_store: 88


In [7]:
for i in range(6):
    get_info_sqllite("bike_store")

Info about: bike_store from sql_lite, now cached with expire=3. Return value=bike_store: 10
Info about: bike_store cached in sql_lite, expire=2. Return value=bike_store: 10
Info about: bike_store cached in sql_lite, expire=1. Return value=bike_store: 10
Info about: bike_store cached in sql_lite, expire=0. Return value=bike_store: 10
Info about: bike_store from sql_lite, now cached with expire=3. Return value=bike_store: 58
Info about: bike_store cached in sql_lite, expire=2. Return value=bike_store: 58


In [8]:
for i in range(10):    
    get_info_postgres("users")

Info about: users from postgres, now cached with expire=5. Return value=users: 43
Info about: users cached in postgres, expire=4. Return value=users: 43
Info about: users cached in postgres, expire=3. Return value=users: 43
Info about: users cached in postgres, expire=2. Return value=users: 43
Info about: users cached in postgres, expire=1. Return value=users: 43
Info about: users cached in postgres, expire=0. Return value=users: 43
Info about: users from postgres, now cached with expire=5. Return value=users: 47
Info about: users cached in postgres, expire=4. Return value=users: 47
Info about: users cached in postgres, expire=3. Return value=users: 47
Info about: users cached in postgres, expire=2. Return value=users: 47


In [9]:
for i in range(6):    
    get_info_sqllite("users")


Info about: users from sql_lite, now cached with expire=3. Return value=users: 8
Info about: users cached in sql_lite, expire=2. Return value=users: 8
Info about: users cached in sql_lite, expire=1. Return value=users: 8
Info about: users cached in sql_lite, expire=0. Return value=users: 8
Info about: users from sql_lite, now cached with expire=3. Return value=users: 51
Info about: users cached in sql_lite, expire=2. Return value=users: 51


# Задача 3 Контекстный менеджер safe_write

In [10]:
import os
class safe_write:
    def __init__(self, filename):
        self.filename = filename

    def __enter__(self):
        self.file = open(self.filename, 'a+')
        self.pos = self.file.tell() # запомнить позицию в файле
        return self.file

    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is not None:
            print(f' ~~~ Во время записи в файл было возбуждено исключение {exc_type.__name__}! ~~~')
            # откатить изменения в файле
            self.file.seek(self.pos, os.SEEK_SET)
            self.file.truncate(self.pos)
            return True
        self.file.close()
        
    def __repr__(self):
        return f'CustomContextManager(value={repr(self.value)})' 
    
def show_file(filename):
    """Показать содержимое файла filename"""
    with open(filename) as file:
        print(f'Содержимое файла {filename}:')
        print(file.read())    

In [11]:
filename = 'undertale.txt'
# сбросить состояние файла в исходное состояние
with open(filename, 'w') as file:
    file.write(f'Инициализация файла \n')    
show_file(filename)

Содержимое файла undertale.txt:
Инициализация файла 



In [12]:
# записать в файл строку с использованием контекстного менеджера safe_write
with safe_write(filename) as file:
    file.write('Я знаю, что ничего не знаю, но другие не знают и этого.\n')
show_file(filename)    

Содержимое файла undertale.txt:
Инициализация файла 
Я знаю, что ничего не знаю, но другие не знают и этого.



In [13]:
# попытаться добавить строку с использованием контекстного менеджера safe_write и сгенерировать при записи исключение
with safe_write(filename) as file:
    file.write('Эта строка не должна сохраниться в файле.\n')
    raise ValueError    
show_file(filename)

 ~~~ Во время записи в файл было возбуждено исключение ValueError! ~~~
Содержимое файла undertale.txt:
Инициализация файла 
Я знаю, что ничего не знаю, но другие не знают и этого.

